In [1]:
# Importing necessary libraries
import pandas as pd

In [2]:
## Upload raw data files

arrival_pax_df = pd.read_csv('data/Arrival Pax Report - Jan2019 to Jan2023.csv')
rideshare_df = pd.read_csv('data/Rideshare_Anon_2023-03-23.csv')
taxi_df = pd.read_csv('data/Taxi_2023-03-31.csv')
weather_df = pd.read_csv('data/climate-daily.csv')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_23212\1547873604.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  taxi_df = pd.read_csv('data/Taxi_2023-03-31.csv')


In [3]:
# Function to calculate percentage of zero values in raw data
def zero_percentage(df):
    zero_count = (df == 0).sum().sum()  # Count of zero values
    total_count = df.size  # Total number of elements
    return (zero_count / total_count) * 100

In [4]:
## Preview of raw data
# Preview and summary for arrival_pax_df
print("Arrival Pax DataFrame:")
print(arrival_pax_df.head())
print("Size (rows, columns):", arrival_pax_df.shape)
print("Percentage of zero values:", zero_percentage(arrival_pax_df))

# Preview and summary for rideshare_df
print("\nRideshare DataFrame:")
print(rideshare_df.head())
print("Size (rows, columns):", rideshare_df.shape)
print("Percentage of zero values:", zero_percentage(rideshare_df))

# Preview and summary for taxi_df
print("\nTaxi DataFrame:")
print(taxi_df.head())
print("Size (rows, columns):", taxi_df.shape)
print("Percentage of zero values:", zero_percentage(taxi_df))

# Preview and summary for weather_df
print("\nWeather DataFrame:")
print(weather_df.head())
print("Size (rows, columns):", weather_df.shape)
print("Percentage of zero values:", zero_percentage(weather_df))


Arrival Pax DataFrame:
          Date Leg         Region Estimated 30 Min  Arr Pax
0  01-Jan-2019   A    Transborder            00:00      221
1  01-Jan-2019   A       Domestic            00:00        0
2  01-Jan-2019   A       Domestic            00:30        3
3  01-Jan-2019   A       Domestic            01:00        3
4  01-Jan-2019   A  Latin America            04:30      160
Size (rows, columns): (110728, 5)
Percentage of zero values: 0.18549960262986778

Rideshare DataFrame:
   Unnamed: 0                                TripID EventType  \
0           1  18ad701d-e458-4799-8658-eaa051010f3b    PickUp   
1           2  fe02f6bc-e02d-4c65-a838-2d45df4cab51    PickUp   
2           3  0bdb7428-e8d0-453b-9cdb-086361e530c1   DropOff   
3           4  143f0ea0-01ff-4aaf-98ef-f18d977cf88b   DropOff   
4           5  90c0152d-3c5b-49f3-bc58-9e7e13644ab2    PickUp   

           PickupDepDT  PickupLat  PickupLng        DropoffDeptDT  DropoffLat  \
0  2021-11-01 14:26:12  49.193794 -123.180

In [5]:
## Cleaning function development
# The clean_dataframe() function is defined to clean a dataframe by dropping rows with NA values in specified columns
# and converting specified columns to datetime format


def clean_dataframe(df, drop_na_columns=None, date_columns=None, drop_zero_rows=None):
    # Drop rows with NA values in specified columns
    if drop_na_columns:
        df_cleaned = df.dropna(subset=drop_na_columns)
    else:
        df_cleaned = df

    # Convert specified columns to datetime format
    if date_columns:
        for col in date_columns:
            df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')
    
    # Drop rows with zero values
    if drop_zero_rows:
        df_cleaned = df_cleaned.loc[(df_cleaned != 0).all(axis=1)]

    return df_cleaned

# Clean DataFrames

arrival_pax_df_cleaned = clean_dataframe(arrival_pax_df, drop_zero_rows=True)
rideshare_df_cleaned = clean_dataframe(rideshare_df, drop_zero_rows=True)
taxi_df_cleaned = clean_dataframe(taxi_df, drop_zero_rows=True)
weather_df_cleaned = weather_df


# Display summary after cleaning
print("Summary after cleaning:")
print("Arrival Pax DataFrame - Size (rows, columns):", arrival_pax_df_cleaned.shape)
print("Rideshare DataFrame - Size (rows, columns):", rideshare_df_cleaned.shape)
print("Taxi DataFrame - Size (rows, columns):", taxi_df_cleaned.shape)
print("Weather DataFrame - Size (rows, columns):", weather_df_cleaned.shape)

Summary after cleaning:
Arrival Pax DataFrame - Size (rows, columns): (109701, 5)
Rideshare DataFrame - Size (rows, columns): (1813180, 11)
Taxi DataFrame - Size (rows, columns): (3673440, 17)
Weather DataFrame - Size (rows, columns): (1561, 36)


In [6]:
## Arrivals data frame development

arrivals = arrival_pax_df_cleaned.copy()

# Convert 'Date' column to datetime format
arrivals['Date'] = pd.to_datetime(arrivals['Date'])

# Extract hour information from 'Estimated 30 Min' column
arrivals['Hour'] = pd.to_numeric(arrivals['Estimated 30 Min'].str[:2], errors='coerce')

# Remove rows with NaN values in 'Hour' column
arrivals = arrivals.dropna(subset=['Hour'])

# Group by 'Date' and 'Hour' columns and sum the 'Arr Pax' values
arrivals_grouped = arrivals.groupby(['Date', 'Hour'])['Arr Pax'].sum().reset_index()

# Print the first few rows of the DataFrame to confirm the changes
print(arrivals_grouped.head())

# Filter the 'Date' column for only the year 2022
arrivals_grouped = arrivals_grouped[pd.to_datetime(arrivals_grouped['Date']).dt.year == 2022]

# Save the filtered DataFrame to a CSV file
output_file_path = "arrivals_date_hour_2022.csv"
arrivals_grouped.to_csv(output_file_path, index=False)

# Print the first few rows of the filtered DataFrame to confirm the changes
print(arrivals_grouped.head())

print("Arrivals DataFrame for 2022")
print(f"DataFrame saved as {output_file_path}")

        Date  Hour  Arr Pax
0 2019-01-01   0.0      224
1 2019-01-01   1.0        3
2 2019-01-01   4.0      160
3 2019-01-01   5.0       74
4 2019-01-01   6.0      578
            Date  Hour  Arr Pax
20740 2022-01-01   0.0       71
20741 2022-01-01   1.0      502
20742 2022-01-01   6.0      438
20743 2022-01-01   7.0     1996
20744 2022-01-01   8.0      706
Arrivals DataFrame for 2022
DataFrame saved as arrivals_date_hour_2022.csv


In [7]:
## Rideshare dataframe development

rideshare = rideshare_df_cleaned.copy()

# Convert 'PickupDepDT' column to datetime format
rideshare['PickupDepDT'] = pd.to_datetime(rideshare['PickupDepDT'])

# Add separate columns for date, time, and hour
rideshare['Date'] = rideshare['PickupDepDT'].dt.date
rideshare['Hour'] = rideshare['PickupDepDT'].dt.hour

# Add a new column named 'Count' with all values equal to 1
rideshare['Rideshare_Count'] = 1

# Group by 'Date' and 'Hour' columns and sum the 'Count' values
rideshare_grouped = rideshare.groupby(['Date', 'Hour'], as_index=False)['Rideshare_Count'].sum()

# Filter the 'Date' column for only the year 2022
rideshare_grouped = rideshare_grouped[pd.to_datetime(rideshare_grouped['Date']).dt.year == 2022]

print("Rideshare DataFrame")
# Print the resulting DataFrame
print(rideshare_grouped.head())

# Save the DataFrame to a CSV file
output_file_path = "rideshare_date_hour_2022.csv"
rideshare_grouped.to_csv(output_file_path, index=False)

print(f"DataFrame saved as {output_file_path}")

Rideshare DataFrame
            Date  Hour  Rideshare_Count
7413  2022-01-01   0.0               21
7414  2022-01-01   1.0               16
7415  2022-01-01   2.0               51
7416  2022-01-01   3.0               22
7417  2022-01-01   4.0                7
DataFrame saved as rideshare_date_hour_2022.csv


In [8]:
## Taxi dataframe development

taxi = taxi_df_cleaned.copy()

# Convert 'Entry.time' column to datetime format
taxi['Entry.time'] = pd.to_datetime(taxi['Entry.time'])

# Extract date component and create a new column 'Date'
taxi['Date'] = taxi['Entry.time'].dt.date

# Filter the DataFrame by the 'Complete.trip' column to keep only 'Yes' values
taxi_filtered = taxi[taxi['Complete.trip'] == 'Yes']

# Rename the column 'Entry.hour' to 'Hour'
taxi_filtered = taxi_filtered.rename(columns={'Entry.hour': 'Hour'})

# Add a new column named 'Count' with all values equal to 1
taxi_filtered['Count'] = 1

# Group by 'Date' and 'Hour' columns and sum the 'Count' values
taxi_grouped = taxi_filtered.groupby(['Date', 'Hour'], as_index=False)['Count'].sum()

# Rename the 'Count' column to 'Taxi_Count'
taxi_grouped = taxi_grouped.rename(columns={'Count': 'Taxi_Count'})

# Filter the 'Date' column for only the year 2022
taxi_grouped = taxi_grouped[pd.to_datetime(taxi_grouped['Date']).dt.year == 2022]

# Select only the desired columns in the final dataframe
taxi_grouped = taxi_grouped[['Date', 'Hour', 'Taxi_Count']]

print("Taxi DataFrame")
# Print the resulting DataFrame
print(taxi_grouped.head())

# Save the DataFrame to a CSV file
output_file_path = "taxi_date_hour_2022.csv"
taxi_grouped.to_csv(output_file_path, index=False)

print(f"DataFrame saved as {output_file_path}")

Taxi DataFrame
             Date  Hour  Taxi_Count
30687  2022-01-01   1.0          66
30688  2022-01-01   2.0          46
30689  2022-01-01   3.0          48
30690  2022-01-01   4.0           4
30691  2022-01-01   5.0           2
DataFrame saved as taxi_date_hour_2022.csv


In [9]:
## Weather dataframe development
#weather_df_cleaned = weather_df
climate_data = weather_df.copy()

# Convert 'LOCAL_DATE' column to datetime format
climate_data['LOCAL_DATE'] = pd.to_datetime(climate_data['LOCAL_DATE'])

# Add a new column named 'Date' with only the date component
climate_data['Date'] = climate_data['LOCAL_DATE'].dt.date

# Select the desired columns
Weather_Dataframe = climate_data[['Date', 'MEAN_TEMPERATURE', 'MIN_TEMPERATURE', 'MAX_TEMPERATURE']]

# Filter the 'Date' column for only the year 2022
Weather_Dataframe = Weather_Dataframe[pd.to_datetime(Weather_Dataframe['Date']).dt.year == 2022]

# Print the first few rows of the DataFrame to confirm the changes
print("Weather DataFrame for 2022")
print(Weather_Dataframe.head())

# Save the DataFrame to a CSV file
output_file_path = "weather_date_2022.csv"
Weather_Dataframe.to_csv(output_file_path, index=False)

print(f"DataFrame saved as {output_file_path}")

Weather DataFrame for 2022
            Date  MEAN_TEMPERATURE  MIN_TEMPERATURE  MAX_TEMPERATURE
1084  2022-01-01              -4.1             -9.4              1.3
1085  2022-01-02               3.1              0.7              5.6
1086  2022-01-03               3.6              1.8              5.4
1087  2022-01-04               2.7              0.3              5.2
1088  2022-01-05              -1.1             -3.5              1.4
DataFrame saved as weather_date_2022.csv


In [10]:
# Load the dataframes from the CSV files
arrivals_df = pd.read_csv('arrivals_date_hour_2022.csv')
rideshare_df = pd.read_csv('rideshare_date_hour_2022.csv')
taxi_df = pd.read_csv('taxi_date_hour_2022.csv')
weather_df = pd.read_csv('weather_date_2022.csv')

In [11]:
# Display the first few rows of each dataframe
print("Arrivals dataframe:")
print(arrivals_df.head())
print("\nRideshare dataframe:")
print(rideshare_df.head())
print("\nTaxi dataframe:")
print(taxi_df.head())
print("\nWeather dataframe:")
print(weather_df.head())

Arrivals dataframe:
         Date  Hour  Arr Pax
0  2022-01-01   0.0       71
1  2022-01-01   1.0      502
2  2022-01-01   6.0      438
3  2022-01-01   7.0     1996
4  2022-01-01   8.0      706

Rideshare dataframe:
         Date  Hour  Rideshare_Count
0  2022-01-01   0.0               21
1  2022-01-01   1.0               16
2  2022-01-01   2.0               51
3  2022-01-01   3.0               22
4  2022-01-01   4.0                7

Taxi dataframe:
         Date  Hour  Taxi_Count
0  2022-01-01   1.0          66
1  2022-01-01   2.0          46
2  2022-01-01   3.0          48
3  2022-01-01   4.0           4
4  2022-01-01   5.0           2

Weather dataframe:
         Date  MEAN_TEMPERATURE  MIN_TEMPERATURE  MAX_TEMPERATURE
0  2022-01-01              -4.1             -9.4              1.3
1  2022-01-02               3.1              0.7              5.6
2  2022-01-03               3.6              1.8              5.4
3  2022-01-04               2.7              0.3              5.2
4  

In [12]:
# Merge dataframes based on index columns (Date and Hour)
merged_df = pd.merge(arrivals_df, rideshare_df, on=['Date', 'Hour'])
merged_df = pd.merge(merged_df, taxi_df, on=['Date', 'Hour'])
merged_df = pd.merge(merged_df, weather_df, on='Date')

In [13]:
# Display the size of the merged dataframe
print("\nSize of Merged dataframe:")
print("Rows:", merged_df.shape[0])
print("Columns:", merged_df.shape[1])

# Display the merged dataframe
print("\nMerged dataframe:")
print(merged_df.head())


Size of Merged dataframe:
Rows: 7203
Columns: 8

Merged dataframe:
         Date  Hour  Arr Pax  Rideshare_Count  Taxi_Count  MEAN_TEMPERATURE  \
0  2022-01-01   1.0      502               16          66              -4.1   
1  2022-01-01   6.0      438                3          24              -4.1   
2  2022-01-01   7.0     1996               17          36              -4.1   
3  2022-01-01   8.0      706                9         106              -4.1   
4  2022-01-01   9.0      952               13          60              -4.1   

   MIN_TEMPERATURE  MAX_TEMPERATURE  
0             -9.4              1.3  
1             -9.4              1.3  
2             -9.4              1.3  
3             -9.4              1.3  
4             -9.4              1.3  


In [14]:
# Save the merged DataFrame to a CSV file
output_file_path = "merged_df.csv"
merged_df.to_csv(output_file_path, index=False)

print(f"\nMerged DataFrame saved as {output_file_path}")


Merged DataFrame saved as merged_df.csv
